# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4ec06941f0>
├── 'a' --> tensor([[ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784]])
└── 'x' --> <FastTreeValue 0x7f4ec06fbe20>
    └── 'c' --> tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                        [-1.2797, -1.1890, -1.4070, -0.8795],
                        [ 1.3458,  0.2088,  1.3505,  1.4118]])

In [4]:
t.a

tensor([[ 0.0700,  1.2872, -0.1570],
        [-0.9522,  0.9468, -0.4784]])

In [5]:
%timeit t.a

57.6 ns ± 1.33 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4ec06941f0>
├── 'a' --> tensor([[-0.4652,  0.3360, -0.4944],
│                   [-1.6189, -1.3267,  1.8182]])
└── 'x' --> <FastTreeValue 0x7f4ec06fbe20>
    └── 'c' --> tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                        [-1.2797, -1.1890, -1.4070, -0.8795],
                        [ 1.3458,  0.2088,  1.3505,  1.4118]])

In [7]:
%timeit t.a = new_value

61.6 ns ± 0.191 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784]]),
    x: Batch(
           c: tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                      [-1.2797, -1.1890, -1.4070, -0.8795],
                      [ 1.3458,  0.2088,  1.3505,  1.4118]]),
       ),
)

In [10]:
b.a

tensor([[ 0.0700,  1.2872, -0.1570],
        [-0.9522,  0.9468, -0.4784]])

In [11]:
%timeit b.a

51 ns ± 0.265 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9628, -0.4238,  1.0286],
               [ 0.9440, -0.0190,  0.0432]]),
    x: Batch(
           c: tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                      [-1.2797, -1.1890, -1.4070, -0.8795],
                      [ 1.3458,  0.2088,  1.3505,  1.4118]]),
       ),
)

In [13]:
%timeit b.a = new_value

492 ns ± 0.541 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

802 ns ± 1.16 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

9.32 µs ± 25.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 3.9 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

128 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4e105b6ac0>
├── 'a' --> tensor([[[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]],
│           
│                   [[ 0.0700,  1.2872, -0.1570],
│                    [-0.9522,  0.9468, -0.4784]]])
└── 'x' --> <FastTreeValue 0x7f4e93f41d90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

29.2 µs ± 61.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4e1afd4eb0>
├── 'a' --> tensor([[ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784],
│                   [ 0.0700,  1.2872, -0.1570],
│                   [-0.9522,  0.9468, -0.4784]])
└── 'x' --> <FastTreeValue 0x7f4e099cf6a0>
    └── 'c' --> tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                        [-1.2797, -1.1890, -1.4070, -0.8795],
                 

In [23]:
%timeit t_cat(trees)

30.1 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.3 µs ± 79.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]],
       
               [[ 0.0700,  1.2872, -0.1570],
                [-0.9522,  0.9468, -0.4784]]]),
    x: Batch(
           c: tensor([[[-1.0942, -0.2883, -0.0287, -1.9157],
                       [-1.2797, -1.1890, -1.4070, -0.8795],
                       [ 1.3458,  0.2088,  1.3505,  1.4118]],
         

In [26]:
%timeit Batch.stack(batches)

69 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784],
               [ 0.0700,  1.2872, -0.1570],
               [-0.9522,  0.9468, -0.4784]]),
    x: Batch(
           c: tensor([[-1.0942, -0.2883, -0.0287, -1.9157],
                      [-1.2797, -1.1890, -1.4070, -0.8795],
                      [ 1.3458,  0.2088,  1.3505,  1.4118],
                      [-1.0942, -0.2883, -0.0287, -1.9157],
                      [-1.2797, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

293 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
